In [8]:
import json
import re
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Funzione per pulire il testo delle email
def clean_email(text):
    text = re.sub(r"(Best regards|Sent from my .*|Thanks,.*|Cheers,.*)", "", text, flags=re.IGNORECASE)
    text = re.sub(r"On .* wrote:", "", text)
    text = re.sub(r"<[^>]+>", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Carica il file JSON con le email
with open("./data/emails.json", "r", encoding="utf-8") as f:
    emails = json.load(f)

# Inizializza il modello
model = SentenceTransformer("all-MiniLM-L6-v2")

# Query di esempio
query = "vorrei trovare la mail che mi permette di trovare informazioni su case in vendita"
query_embedding = model.encode(query, convert_to_tensor=True)

# Calcolo delle similarità
similarities_raw = []
similarities_clean = []

for email in emails:
    text = email.get("body", "")
    cleaned_text = clean_email(text)

    embedding_raw = model.encode(text, convert_to_tensor=True)
    embedding_clean = model.encode(cleaned_text, convert_to_tensor=True)

    sim_raw = util.cos_sim(query_embedding, embedding_raw).item()
    sim_clean = util.cos_sim(query_embedding, embedding_clean).item()

    similarities_raw.append(sim_raw)
    similarities_clean.append(sim_clean)

# Statistiche
avg_raw = np.mean(similarities_raw)
avg_clean = np.mean(similarities_clean)

print(f"Similarità media (originali): {avg_raw:.4f}")
print(f"Similarità media (pulite): {avg_clean:.4f}")

# Top 3 email
top_raw_indices = np.argsort(similarities_raw)[-3:][::-1]
top_clean_indices = np.argsort(similarities_clean)[-3:][::-1]

print("\nTop 3 email (originali):")
for i in top_raw_indices:
    print(f"- Similarità: {similarities_raw[i]:.4f} | Estratto: {emails[i]['body'][:100]}...")

print("\nTop 3 email (pulite):")
for i in top_clean_indices:
    print(f"- Similarità: {similarities_clean[i]:.4f} | Estratto: {clean_email(emails[i]['body'])[:100]}...")


Similarità media (originali): 0.1531
Similarità media (pulite): 0.1535

Top 3 email (originali):
- Similarità: 0.5559 | Estratto: Verifica la tua email
 
Dobbiamo verificare il tuo indirizzo email marcofrancescosommaruga@gmail.c...
- Similarità: 0.3569 | Estratto: Accedi a Streamlit Community Cloud
 
Hai richiesto di accedere a Streamlit Community Cloud. Il tuo...
- Similarità: 0.3141 | Estratto: Buongiorno,

Chiedo conferma che i documenti inviati siano ora leggibili e novità sui tempi di res...

Top 3 email (pulite):
- Similarità: 0.5559 | Estratto: Verifica la tua email Dobbiamo verificare il tuo indirizzo email marcofrancescosommaruga@gmail.com p...
- Similarità: 0.3569 | Estratto: Accedi a Streamlit Community Cloud Hai richiesto di accedere a Streamlit Community Cloud. Il tuo cod...
- Similarità: 0.3103 | Estratto: Buongiorno, Chiedo conferma che i documenti inviati siano ora leggibili e novità sui tempi di reset ...
